## INSTALLING THE NECESSARY DEPENDENCIES:

In [1]:
##(install and restart runtime)
#!pip install --upgrade accelerate
#!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00


In [1]:
import torch
import numpy as np
import pandas as pd

!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import os

from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

torch.__version__


from huggingface_hub import HfFolder, notebook_login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempt

In [2]:
notebook_login()

In [3]:
repository_id = "Geee05/overlapwindow"

## DATA PRE-PROCESSING

In [4]:
dataset = pd.read_csv('/content/All_transcript_train.csv')
tdataset= pd.read_csv('/content/All_transcript_test_new.csv')

In [5]:
dataset.rename(columns = {'Class':'labels'}, inplace = True)
tdataset.rename(columns = {'Group':'labels'}, inplace = True)

In [6]:
dataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
tdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)

In [7]:
dataset['labels'].value_counts()

0    126
1     37
Name: labels, dtype: int64

In [8]:
#resample data to deal with class imbalance
from sklearn.utils import resample
df_1=dataset[dataset['labels']==1]
df_2=dataset[dataset['labels']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=126,replace=True)
df=pd.concat([df_upsampled,df_2])
df.reset_index(inplace=True)

In [9]:
#creating Datasets
train = Dataset.from_pandas(df, preserve_index=False)
test = Dataset.from_pandas(tdataset, preserve_index=False)

print(train)
print(test)

Dataset({
    features: ['index', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 252
})
Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 56
})


In [10]:
leng=[]
for i in range (0,len(train['Text'])):
  leng.append(len(train['Text'][i]))

print(min(leng))
print(max(leng))

1362
21064


### Function Definition:

In [11]:
#SEGMENT TEXTS
def segment_function(example):
  seg=example['Text']
  res=[]
  n=1000

  for iter in range (0,len(seg)):
    for i in range (0,len(seg[iter])):
      j=i*500

      if j<=len(seg[iter]):
        part=(seg[iter][j:j+n])
      else:
        break

      res.append(part)

  return res

#TOKENIZE TEXTS
tokenizer = AutoTokenizer.from_pretrained('gooohjy/suicidal-electra')
def tokenize_function(example):
  return tokenizer(example["Text"], padding='max_length',truncation=True,max_length=512,add_special_tokens = True)

In [12]:
#LABELS CORR. TO SEGMENTS
label=[]
for i in range (0,len(train)):
  x = ((len(train['Text'][i]))//500) + 1
  for j in range (0,x):
    label.append(train['labels'][i])


label_t=[]
for i in range (0,len(test)):
  x = ((len(test['Text'][i]))//500) + 1
  for j in range (0,x):
    label_t.append(test['labels'][i])

#### Segmenting Train Data

In [13]:
ip_seg=segment_function(train)

##### (checking overlap):

In [14]:
ip_seg[5]

"ertain areas of life that I'm not comfortable with south of the few things that he'll be nice so I can erase it completely  yeah  hurting people I didn't need to be hurt emotionally especially  few women in my life are my past  probably put my mother through certain things that shouldn't have to go through along with my close friends and family as well  tell me about your relationship with your family  used to be Rocky growing up but as I got older mature and Discerning spirits that I've been through kind of brought me closer to my family so now it's it is pretty decent this is pretty soon  I would say my mother  yeah mother  oh she she she had her share experiences negative and positive I might her the struggle she been through  she she she has experienced difficult economic hardship growing up but that didn't stop her for raising a son at a young age finishing school getting her Masters in just just overall being a good person and she also about her mental health issues as well and 

In [15]:
ip_seg[6]

"rough kind of brought me closer to my family so now it's it is pretty decent this is pretty soon  I would say my mother  yeah mother  oh she she she had her share experiences negative and positive I might her the struggle she been through  she she she has experienced difficult economic hardship growing up but that didn't stop her for raising a son at a young age finishing school getting her Masters in just just overall being a good person and she also about her mental health issues as well and probably not the best of family that she could have been raised with social so she's kind of like a inspiration to to me and other people as well so I Was Your Mother  how easy is it for you to get a good night sleep  it's not easy so yeah it's not easy  I can tell you I've always had sleeping issues why you should just take like over counter sleep medication or NyQuil but it makes you drowsy the next morning so it looks like it's a negative effect really  the medicate I don't know shitt I don't

In [16]:
#preparing dataframe for segmented texts & corresponding labels
train_data=pd.DataFrame(ip_seg,columns=['Text'])
train_data['labels']=label
train_data.head(2)

,Text,labels
0,okay hi I'm Ellie thanks for coming in today ...,1
1,time just I guess you know stay focused Lil ...,1


### Tokenizing Train Data

In [17]:
trainset = Dataset.from_pandas(train_data, preserve_index=False)
trainset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 3592
})

In [18]:
tokenized_train_dataset = trainset.map(tokenize_function, batched=True)
tokenized_train_dataset

Map:   0%|          | 0/3592 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3592
})

#### Segmenting Test Data

In [19]:
ip_seg_t=segment_function(test)

test_data=pd.DataFrame(ip_seg_t,columns=['Text'])
test_data['labels']=label_t
test_data.head(2)

,Text,labels
0,okay there she is coming to go ahead and Shrin...,0
1,my friends and we should have fun I'm very cl...,0


### Tokenizing Test Data

In [20]:
testset = Dataset.from_pandas(test_data, preserve_index=False)
testset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 663
})

In [21]:
tokenized_test_dataset = testset.map(tokenize_function, batched=True)
tokenized_test_dataset

Map:   0%|          | 0/663 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 663
})

### Review

In [22]:
#tokenizer.model_max_length =512

In [23]:
print(len(label))
print(len(label_t))


3592
663


In [24]:
print(len(ip_seg))
print(len(ip_seg_t))


3592
663


In [25]:
type(tokenized_train_dataset)

datasets.arrow_dataset.Dataset

## TRAINING

In [26]:
# Set dataset format using set_format() function to specify the dataset format, making it compatible with PyTorch
tokenized_train_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
tokenized_test_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [27]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from sklearn.metrics import classification_report

training_args = TrainingArguments(evaluation_strategy="epoch",num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    output_dir=repository_id,
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token())   # default arguments for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('gooohjy/suicidal-electra', num_labels=2) #,ignore_mismatched_sizes=True
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    #probabilities = softmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
    print((classification_report(labels,predictions)))

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Cloning https://huggingface.co/Geee05/overlapwindow into local empty directory.


Download file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

Download file logs/events.out.tfevents.1688285397.24a42a5015d8.1475.0:  30%|###       | 16.5k/54.6k [00:00<?, …

Download file logs/events.out.tfevents.1688123122.8db7aef813e9.36050.0: 100%|##########| 9.00k/9.00k [00:00<?,…

Download file logs/events.out.tfevents.1688115927.8db7aef813e9.3526.0: 100%|##########| 6.18k/6.18k [00:00<?, …

...

Clean file logs/events.out.tfevents.1688115927.8db7aef813e9.3526.0:  16%|#6        | 1.00k/6.18k [00:00<?, ?B/…

Clean file logs/events.out.tfevents.1688123122.8db7aef813e9.36050.0:  11%|#1        | 1.00k/9.00k [00:00<?, ?B…

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file logs/events.out.tfevents.1688285397.24a42a5015d8.1475.0:   2%|1         | 1.00k/54.6k [00:00<?, ?B/…

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.501800,0.737342,0.634992,0.514056
2,0.395800,1.945347,0.624434,0.437923
3,0.156900,2.080769,0.674208,0.428571
4,0.000200,2.354230,0.722474,0.452381
5,0.000100,2.464436,0.698341,0.404762
6,0.000000,2.942366,0.678733,0.466165
7,0.000000,2.769261,0.720965,0.346290
8,0.000000,3.142920,0.680241,0.464646
9,0.000000,3.146906,0.687783,0.448000
10,0.000100,3.000541,0.708899,0.420420


<ipython-input-32-84615fee8da7>:21: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=4490, training_loss=0.1413504857550941, metrics={'train_runtime': 3988.9738, 'train_samples_per_second': 9.005, 'train_steps_per_second': 1.126, 'total_flos': 9450949108531200.0, 'train_loss': 0.1413504857550941, 'epoch': 10.0})

## PREDICTION

In [ ]:
predict=trainer.predict(tokenized_test_dataset)

In [ ]:
pred_labels=predict[1]

In [ ]:
true_labels=np.array(testset['labels'])

In [ ]:
type(predict)

transformers.trainer_utils.PredictionOutput

## SAVING THE MODEL

In [ ]:
# Save our tokenizer and create a model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

Upload file logs/events.out.tfevents.1688376900.f97373aa76c3.10963.0:   0%|          | 1.00/76.8k [00:00<?, ?B…

To https://huggingface.co/Geee05/overlapwindow
   4c7e4cc..573d127  main -> main

   4c7e4cc..573d127  main -> main



'https://huggingface.co/Geee05/overlapwindow/commit/573d1278802a7fd5b2a2a24d5168a16f1a0dde39'

## VALIDATION

In [ ]:
vdataset= pd.read_csv('/content/All_transcript_val_new.csv')

In [ ]:
vdataset.head(2)

,Unnamed: 0.1,Unnamed: 0,File,Text,Group
0,0,0,600_AUDIO,okay there she is coming to go ahead and Shrin...,HC
1,1,1,602_AUDIO,this is super need I like this me either at a...,DP


In [ ]:
text=vdataset['Text'][1]

In [ ]:
vdataset.rename(columns = {'Group':'labels'}, inplace = True)
vdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
vdataset['labels'].value_counts()

0    39
1    17
Name: labels, dtype: int64

In [ ]:
val = Dataset.from_pandas(vdataset, preserve_index=False)
print(val)

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 56
})


In [ ]:
#LABELS CORR. TO SEGMENTS

label_v=[]
for i in range (0,len(val)):
  x = ((len(val['Text'][i]))//500) + 1
  for j in range (0,x):
    label_v.append(val['labels'][i])

In [ ]:
ip_seg_v=segment_function(val)

val_data=pd.DataFrame(ip_seg_v,columns=['Text'])
val_data['labels']=label_v
val_data.head(2)

,Text,labels
0,okay there she is coming to go ahead and Shrin...,0
1,my friends and we should have fun I'm very cl...,0


In [ ]:
valset = Dataset.from_pandas(val_data, preserve_index=False)
valset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 663
})

In [ ]:
tokenized_val_dataset = valset.map(tokenize_function, batched=True)
tokenized_val_dataset

Map:   0%|          | 0/663 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 663
})

In [ ]:
tokenized_val_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [ ]:
tokenized_val_dataset

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 663
})

### (calling saved model via pipelining):

In [ ]:
from transformers import pipeline

classifier = pipeline('text-classification',repository_id)

result=[]
for i in range (0,len(tokenized_val_dataset['Text'])):

  res=(classifier(tokenized_val_dataset['Text'][i]))
  predicted_label = res[0]["label"]
  result.append(predicted_label)

In [ ]:
type(result)

list

#### (processing the results to desired format):

In [ ]:
df=pd.DataFrame(result,columns=['result'])
df['result']=df['result'].str.replace('\D', '', regex=True)
result=df['result']

In [ ]:
df['result']=df['result'].astype(int)
result=df['result']

In [ ]:
true_labels=[]
for i in range (0,len(tokenized_val_dataset['labels'])):
  true_labels.append(tokenized_val_dataset['labels'][i])


## CLASSIFICATION REPORT

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels,result))

              precision    recall  f1-score   support

           0       0.80      0.63      0.71       462
           1       0.43      0.64      0.51       201

    accuracy                           0.63       663
   macro avg       0.62      0.64      0.61       663
weighted avg       0.69      0.63      0.65       663



## MISCELLANEOUS


In [ ]:
sentiment=pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
sentiment("okay there she is coming to go ahead and Shrink her back down while I continue setting something's out okay  when she says goodbye  okay  yes  I'm fine  Mexico  the weather  the crime  I don't know what that means  write poetry  nothing  mr. music  Burger  about a week ago my husband over cigarettes  she wants to stop and I don't  I dropped out when I was in 11th grade  I'm on the Floyd  I would like to become a famous poetry writer  yeah  no  so expensive  when I was little I used to play with my friends and we should have fun  I'm very close  yeah  my husband  can you tell me about  well  I used to be a drug addict and he'll help me overcome my addiction  and I feel better  it's fine  no just my husband  no  no  yes  2002  no really  overdose on pills  I'd love to  no  excuse me  I'm not attending therapy right now  I'm not going to be right now I don't go  I'm not comfortable talking about this  I don't feel comfortable talking about this  I helped me a lot  Shadow time  no one to do anything  sleep too much  oh no  erotic poetry helps me a lot  well  when I get inspired I just start right riding what's on my mind  and I like it  good  awesome  yes it's been a big change I feel happy hour now  more outgoing  we are  yes  no  very easy  I just go lay down and just to the music for a little while and you'll fall asleep  cranky  today  I don't know I'm just in a happy mood today  a sappy person  outgoing  nothing for high school  I'm just happy today")

[{'label': 'POSITIVE', 'score': 0.8562464714050293}]

In [ ]:
sentiment("I don't know sometimes therapies helpful sometimes it's not  show the happiness that was a question sometimes I've had half my happiest moment was when I climbed up inside the Great Pyramid of Giza that was extremely hot I don't know it's just wonderful  wonderful feeling I'm claustrophobic so I was proud that I made it through there and it was just an amazing sight  Cassius is a test to see how one reacts when there's an uncomfortable silence  am I supposed to be pushing the button for the person to come in now  all right she did stop talking  I thought it was a test of how one reacts during an uncomfortable side")

[{'label': 'POSITIVE', 'score': 0.9952889680862427}]